## Testing and comparing thermobarometers on an experimental dataset
- This notebook was developed for the 2024 USGS Petrology workshop
- It shows how to perform Cpx-only, Cpx-Liq and Liq-only based thermobarometry calculations on new experimental test data compiled in Wieser et al. (2023) - https://doi.org/10.31223/X59655
- Please try running up to cell 3 before the tutorial!

In [ ]:
# First, you need to install Thermobar - You only need to run this once per computer - once you've run it, add a # infront!
#!pip install Thermobar

In [ ]:
# Now import it and check version
import Thermobar as pt
print(pt.__version__)
# lets import other python things too
import numpy as np
# Pandas is like spreadsheets in python
import pandas as pd
# Lets get it to display all pandas columns
pd.set_option('display.max_columns', None)
# This is for plotting
import matplotlib.pyplot as plt

In [ ]:
# Lets load the data - You need to make sure the excel spreadsheet is in the same folder as the notebook. 
out=pt.import_excel('ArcPL_Filtered_Dataset.xlsx', sheet_name='Sheet1')
# This loads all columns you have inputted
all_input=out['my_input']
# This pulls out just Cpxs - E.g. columns with _Cpx and _Liq after
Cpx=out['Cpxs']
Liq=out['Liqs']

In [ ]:
# You should also check that things have loaded in how you expect - e.g. looking for columns full of zeros!
Cpx.head()

In [ ]:
# Lets check the liquids too
Liq.head()

## Lets calculate some liquid-only temperatures first

In [ ]:
# We find that this adapted ol-liq thermometer that uses calculated DMg values from Beattie is the best all around performer!
# Here, we are using a pressure of 5 kbar
Temp_Eq22_Put=pt.calculate_liq_only_temp(liq_comps=Liq, equationT='T_Put2008_eq22_BeattDMg', P=5)
Temp_Eq22_Put

In [ ]:
# If you love excel, you can copy at this point to your clipboard!
# This wont work in binder
Temp_Eq22_Put.to_clipboard(excel=True, index=False)

In [ ]:
# Or simply append onto your dataframe, and save
# Adding the new column
all_input['Temp_Puteq22']=Temp_Eq22_Put
# Saving to excel
all_input.to_excel('USGS_thermobarometry_output.xlsx')

In [ ]:
# Try with your fav equation
help(pt.calculate_liq_only_temp)

## Perform calculations  by iterating a thermometer and a barometer
- How do you know what options are in Thermobar? You can look at the help function! It tells you what all the different equations are called

## Calc 1 - Using Neave and Putirka (2017) and Putirka (2008) eq33

In [ ]:
# This calculation uses pre-matched Cpx and Liq pairs from these experiments
Neave17=pt.calculate_cpx_liq_press_temp(cpx_comps=Cpx, liq_comps=Liq, equationP='P_Neave2017',
                                   equationT='T_Put2008_eq33')
Neave17.head()

### Lets compare this to experimental pressures!
- you can check out columns you loaded in using .head()

In [ ]:
# Remember, all_input was the thing returned that had all your entered columns - 'P_kbar_x' gives the experimental pressure
all_input.head()

In [ ]:
# This line sets up a plot with 1 subplot, that is 6 units by 5 units in size. 
fig, (ax1) = plt.subplots(1, 1, figsize=(6,5))
# This plots experimental pressure vs. Calculated pressure
ax1.plot(all_input['P_kbar_x'], Neave17['P_kbar_calc'], '.k')
# Now add a 1:1 line to compare!
ax1.plot([0, 15], [0, 15], '-r')
# Here is how you add a Y label - Try adding an X
ax1.set_ylabel('Calc P kbar (NP17-Eq33)')
# Add your own x label here! - X is experimental pressure



### Now do the same for Temperature 

In [ ]:
# Try typing code here to make a plot! You need to replace P_kbar with T_K at every point



### Scroll down for answers (only after you have tried!)
-
-
-
-
-
-
-
-
-
-
-
-


In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,5))
ax1.plot(all_input['P_kbar_x'], Neave17['P_kbar_calc'], '.k')
ax1.plot([0, 17], [0, 17], '-r')
ax1.set_xlabel('Experiment P (kbar)')
ax1.set_ylabel('Calc P (kbar)')

ax2.plot(all_input['T_K_x']-273.15, Neave17['T_K_calc']-273.15, '.k')
ax2.plot([800, 1300], [800, 1300], '-r')
ax2.set_xlabel('Experiment T (°C)')
ax2.set_ylabel('Calc T (°C)')


### What can we say about the relative success of thermobarometers vs. barometers? What does this maybe mean thermodynamically?

In [ ]:
## What if we want to calculate some statistics? Here, we can do a linear regression between exp pressure vs. calc pressure
pt.calculate_R2(all_input['P_kbar_x'], Neave17['P_kbar_calc'], xy=False)

### Now do the same calculation for temperature!

In [ ]:
# write some code here!


### What does a RMSE of 3.92 kbar mean in terms of depth in the crust (choose your favourite crustal density value!)



In [ ]:
# type your working here! Remember, P = rho * g *H


### Scroll down for an inbuilt function to do this (once you've had a go!)
-
-
-
-
-
-
-
-
-
-
-
-


In [ ]:
# Lets use a fixed crustal density profile
Depth1=pt.convert_pressure_to_depth(P_kbar=3.92, crust_dens_kgm3=2700)
Depth1

In [ ]:
help(pt.convert_pressure_to_depth)

In [ ]:
# Lots of different options for profiles with the pressures we calculated with Neave
Depth_P=pt.convert_pressure_to_depth(P_kbar=Neave17['P_kbar_calc'], 
                                        crust_dens_kgm3='prezzi')
Depth_P

## Calc 2 - Now try calculations using Putirka (2003) for both T and P. Read the documentation above to find out what these are called. Or go here 
- https://github.com/PennyWieser/Thermobar/blob/main/docs/img/All_Phases_Docs_Merged.pdf

In [ ]:
# how do the Ts compare to the ones above with eq33?


## Calc 3 - But how do we know if the Cpx-Liq pairs are actually in equilibrium?
- You can ask Thermobar to perform the common equilibrium tests for you! 

In [ ]:
Neave17_Eqtest=pt.calculate_cpx_liq_press_temp(cpx_comps=Cpx, liq_comps=Liq, equationP='P_Neave2017',
                                   equationT='T_Put2008_eq33', eq_tests=True)
Neave17_Eqtest.head()

### How would we filter based on Kd within 0.03?

In [ ]:
# Firts lets look at the distribution of equilibrium tests
plt.hist(Neave17_Eqtest['Delta_Kd_Put2008'])
plt.xlabel('Kd Meas - Kd Putirka eq35')
plt.ylabel('# of analyses')

In [ ]:
# This makes a filtered dataframe, taking only the rows where Delta Kd is < or equal to 0.03
Filtered_KD=Neave17_Eqtest.loc[Neave17_Eqtest['Delta_Kd_Put2008']<=0.03]
Filtered_KD.head()

## Calc 4 - how do we compare lots of equations?
- Sure, you could type out every combination of equation, but thats sort of tedious! We have included the most common combinations used in the literature!
- It doesnt return equilibrium tests, because these have terms for P and T, so depends what equations you choose! 

### Some of these are machine learning equations - we need to download these models separatly as they are rather big and PyPI gets annoyed at us (its a free service!)


In [ ]:
# Check your version of sklearn - you may need to upgrade if you are on a version earlier than 1.3
import sklearn as s
s.__version__

In [ ]:
# Now install the machine learning files
!pip install "https://github.com/PennyWieser/Thermobar_onnx/archive/refs/tags/v.0.0.4.zip"

In [ ]:
All_Exps=pt.calculate_cpx_liq_press_all_eqs(cpx_comps=Cpx, liq_comps=Liq)
All_Exps.head()

### Choose your 3 favourite equations for pressure - and plot them against experimental pressure here


In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(12,4))
# Add the rest of the code here - you can paste from above

## Calc 5 - What can we do in python that is hard elsewhere?
- We can plot our own data amongst the calibration datasets of different models! 

In [ ]:
Cali_Dataset_Brugman=pt.return_cali_dataset(model="Brugman2019")
Cali_Dataset_Neave=pt.return_cali_dataset(model="Neave2017")
Cali_Dataset_Wang=pt.return_cali_dataset(model="Wang2021")
Cali_Dataset_Masotta=pt.return_cali_dataset(model="Masotta2013")
Cali_Dataset_Putirka=pt.return_cali_dataset(model="Putirka2008")

In [ ]:
Cali_Dataset_Brugman

In [ ]:
# If we want to draw a TAS diagram, we can import from here
!pip install git+https://bitbucket.org/jsteven5/tasplot.git


## We need to import TAS plot

In [ ]:
import tasplot

In [ ]:
#fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10,10), sharey=True)
figure_mosaic="""
AA
BC
"""
fig, axes=plt.subplot_mosaic(mosaic=figure_mosaic, figsize=(8,8))
# Plot the calibration dataset of different models
axes['A'].plot(Cali_Dataset_Putirka['SiO2_Liq'], Cali_Dataset_Putirka['Na2O_Liq']+Cali_Dataset_Putirka['K2O_Liq'],
         's', mec='grey', mfc='grey', ms=2, label='P08')
axes['A'].plot(Cali_Dataset_Brugman['SiO2_Liq'], Cali_Dataset_Brugman['Na2O_Liq']+Cali_Dataset_Brugman['K2O_Liq'],
         'sk', mfc='cyan', label='B19')
axes['A'].plot(Cali_Dataset_Wang['SiO2_Liq'], Cali_Dataset_Wang['Na2O_Liq']+Cali_Dataset_Wang['K2O_Liq'],
         'oy', mfc='yellow', ms=3, label='W21')
axes['A'].plot(Cali_Dataset_Neave['SiO2_Liq'], Cali_Dataset_Neave['Na2O_Liq']+Cali_Dataset_Neave['K2O_Liq'],
         'dk', mfc='red', label='N17')
axes['A'].plot(Cali_Dataset_Masotta['SiO2_Liq'], Cali_Dataset_Masotta['Na2O_Liq']+Cali_Dataset_Masotta['K2O_Liq'],
          'pb', mfc='None', ms=4, mew=1, label='M13')
axes['A'].set_xlim([35, 80])
axes['A'].set_xlabel('SiO$_2$ (wt%)')
axes['A'].set_ylabel('Na$_2$O + K$_2$O (wt%)')
axes['A'].legend()

###
axes['B'].plot(Cali_Dataset_Putirka['Mgno_Cpx'], Cali_Dataset_Putirka['Al2O3_Cpx'],
         's', mec='grey', mfc='grey', ms=2)
axes['B'].plot(Cali_Dataset_Brugman['Mgno_Cpx'], Cali_Dataset_Brugman['Al2O3_Cpx'],
         'sk', mfc='cyan')
axes['B'].plot(Cali_Dataset_Wang['Mgno_Cpx'], Cali_Dataset_Wang['Al2O3_Cpx'],
         'oy', mfc='yellow', ms=3)
axes['B'].plot(Cali_Dataset_Neave['Mgno_Cpx'], Cali_Dataset_Neave['Al2O3_Cpx'],
         'dk', mfc='red')
axes['B'].plot(Cali_Dataset_Masotta['Mgno_Cpx'], Cali_Dataset_Masotta['Al2O3_Cpx'],
         'pb', mfc='None', ms=4, mew=1)
#axes['B'].set_xlim([35, 80])
axes['B'].set_xlabel('Mg# Cpx')
axes['B'].set_ylabel('Al$_2$O$_3$ Cpx (wt%)')



axes['C'].plot(Cali_Dataset_Putirka['T_K']-273.15, Cali_Dataset_Putirka['P_kbar'],
         's', mec='grey', mfc='grey', ms=2)
axes['C'].plot(Cali_Dataset_Brugman['T_K']-273.15, Cali_Dataset_Brugman['P_kbar'],
         'sk', mfc='cyan')
axes['C'].plot(Cali_Dataset_Wang['T_K']-273.15, Cali_Dataset_Wang['P_kbar'],
         'oy', mfc='yellow', ms=3)
axes['C'].plot(Cali_Dataset_Neave['T_K']-273.15, Cali_Dataset_Neave['P_kbar'],
         'dk', mfc='red')
axes['C'].plot(Cali_Dataset_Masotta['T_K']-273.15, Cali_Dataset_Masotta['P_kbar'],
         'pb', mfc='None', ms=4, mew=1)

axes['C'].set_ylim([-1, 20.5])
axes['C'].set_xlim([600, 1500])
axes['C'].set_xlabel('T (°C)')

axes['C'].set_ylabel('P (kbar)')


# This is the plot that actually does the TAS lines
tasplot.add_LeMaitre_fields(axes['A'])
for t in axes['A'].texts[:]:
    t.remove()
    #ax1.remove_artist(t)

fig.tight_layout()
fig.savefig('Cali_Dataset_Cpx.png', dpi=200)